In [1]:
import os

In [2]:
from langchain_ollama import ChatOllama, OllamaLLM

llm_llama3 = ChatOllama(
    model="llama3.2",
    temperature=0,
    # other params...
)

In [4]:
from uuid import uuid4
from langchain_core.documents import Document

In [5]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

sntnc_trnsfrmr_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vector_store = Chroma(
    embedding_function=sntnc_trnsfrmr_embeddings,
)

# vectorstore = FAISS.from_texts(
#     ["harrison worked at kensho"], embedding=sntnc_trnsfrmr_embeddings
# )

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
    id=1,
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
    id=2,
)

retriever = vector_store.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
# model = ChatOpenAI()


In [12]:
documents = [
    document_1,
    document_2,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['047270fc-3714-456f-9714-1ef2c017eea0',
 '93d77882-ad8d-4a69-a3cc-efa643c04835']

In [7]:
from langchain_core.runnables import RunnablePassthrough

# Prompt
template = """Answer the question based only on the following context, which can include text and tables:
{context}
Question: {question}
"""
rag_prompt_text = ChatPromptTemplate.from_template(template)

In [9]:
def text_rag_chain(retriever):
    """RAG chain"""

    # LLM
    # model = ChatOpenAI(temperature=0, model="gpt-4")

    # RAG pipeline
    # chain = (
    #     {"context": retriever, "question": RunnablePassthrough()}
    #     | rag_prompt_text
    #     | llm_llama3
    #     | StrOutputParser()
    # )

    generation_chain = rag_prompt_text | llm_llama3 | StrOutputParser()
    retrieval_chain = {"context": retriever, "question": RunnablePassthrough()
                       } | RunnablePassthrough.assign(output=generation_chain)
 
    return retrieval_chain

In [18]:

retrieval_chain = (
    {"context": vector_store.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | llm_llama3
    | StrOutputParser()
)

retrieval_chain.invoke("where did harrison work?")

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


'There is no mention of a person named Harrison in the provided context. The documents only contain information about breakfast food and a weather forecast.'

In [17]:
retrieval_chain = {"context": vector_store.as_retriever(), "question": RunnablePassthrough()} | RunnablePassthrough.assign(output = (
    prompt | llm_llama3 | StrOutputParser()))

retrieval_chain.invoke("where did harrison work?")

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


{'context': [Document(id='047270fc-3714-456f-9714-1ef2c017eea0', metadata={'source': 'tweet'}, page_content='I had chocolate chip pancakes and scrambled eggs for breakfast this morning.'),
  Document(id='93d77882-ad8d-4a69-a3cc-efa643c04835', metadata={'source': 'news'}, page_content='The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.')],
 'question': 'where did harrison work?',
 'output': 'There is no mention of a person named Harrison in the provided context. The documents only contain information about breakfast food and a weather forecast.'}

In [10]:
chain_baseline = text_rag_chain(retriever)

In [11]:
chain_baseline.invoke("what is the Overall Temp. Range of CI2A K Thermocouple?")

{'context': [],
 'question': 'what is the Overall Temp. Range of CI2A K Thermocouple?',
 'output': 'I don\'t see any information about the "CI2A K Thermocouple" in the provided context. Can you please provide more details or clarify where to find this information? I\'ll do my best to help.'}